In [1]:
%pylab inline
%config InlineBackend.figure_format = 'retina'
style.use('dark_background')
from IPython.display import clear_output
from copy import copy, deepcopy
from tqdm import tqdm_notebook as tm
from tqdm import trange
from glob import glob
import os
import sys

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import pandas.io.sql as psql
import kts
from kts import *
features.define_in_scope(globals())
helpers.define_in_scope(globals())

In [3]:
train = kts.load('train')

In [4]:
@helper
def get_conn():
    import psycopg2 as pg
    return pg.connect("dbname=vk_data user=vk_parser password=WegEWWXfedLf2YbS host=13.84.188.132")


In [19]:
@helper
def run_sql(query):
    import pandas.io.sql as psql
    res = psql.read_sql(query, get_conn())
    try:
        return res.set_index('uid')
    except:
        import warnings
        warnings.warn("[run_sql] UID wasn't selected, default index used")
        return res

In [37]:
@helper
def get_uid():
    return run_sql(f"SELECT uid FROM users WHERE data_loaded")

In [ ]:
# train = get_uid()
# kts.save('train')

In [43]:
@helper
def get_photo_urls():
    return run_sql("SELECT uid, photo_100, photo_200_orig FROM users WHERE data_loaded")

In [46]:
# @preview(train, [10, 20])
@register
def photo_urls(df):
    tmp = get_photo_urls()
    return tmp.loc[df.index]

In [56]:
@helper
def download_img(a):
    if len(a.shape) > 1:
        return
    for field in ['photo_100', 'photo_200_orig']:
        if not os.path.exists(f'../input/{field}'):
            os.mkdir(f'../input/{field}')
        pathsave = f"../input/{field}/{a['uid']}.jpg"
        if os.path.exists(pathsave):
            continue
        with open(pathsave, 'wb') as f:
            _ = f.write(r.get(a[field]).content)
    return a

In [74]:
@helper
def write_embedding(emb):
    emb_col_names = [f'face_{i}' for i in range(128)]
    tmp = pd.Series()
    for col, val in zip(emb_col_names, emb):
        tmp[col] = val
    return tmp

In [75]:
@helper
def get_embedding(row):
    uid = row['uid']
    import face_recognition as fr
    try:
        img = fr.load_image_file(f"../input/photo_200_orig/{uid}.jpg")
        embs = fr.face_encodings(img)
        if len(embs) != 1:
            return write_embedding([None] * 128)
        return write_embedding(embs[0])
    except Exception as e:
        return write_embedding([None] * 128)

In [81]:
# %%time
# @preview(train, [10000])
@register
def embeddings(df):
    tmp = photo_urls(df)
    stl.apply(tmp.reset_index(), download_img, axis=1, n_threads=40)
    
    res = stl.apply(df.reset_index()[['uid']], get_embedding, axis=1, n_threads=40)
    res.index = df.index
    return res
    

In [98]:
# %%time
# @preview(train, [10])
@register
def has_embedding(df):
    tmp = embeddings(df)
    res = stl.empty_like(df)
    res['has_embedding'] = tmp['face_0'].isna()
    return res